# Agricultural Application: Crop Disease Diagnosis

## Real-World Problem

**Scenario**: You manage a large farm and need to decide whether to treat fields for disease based on test results and symptoms. Wrong decisions are costly:
- **False positive**: Waste money treating healthy fields
- **False negative**: Lose crops to untreated disease

This notebook demonstrates using **Bayes' theorem** for optimal disease management.

### Learning Objectives
- Apply conditional probability to real diagnosis
- Understand sensitivity, specificity, and base rates
- Make cost-effective treatment decisions
- Handle uncertainty in agricultural management

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from scipy import stats
import sys
sys.path.append('../2_from_scratch')
import probability_functions as pf

plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
np.random.seed(42)

print("✓ Ready to solve real agricultural problems!")

## Problem 1: Disease Testing with Bayes' Theorem

### Given Information
- **Disease prevalence**: 8% of fields in your region have Fusarium wilt
- **Test sensitivity**: 92% (detects disease when present)
- **Test specificity**: 90% (correct negative when healthy)
- **Treatment cost**: $150/acre
- **Crop loss if untreated**: $800/acre

In [ ]:
# Disease parameters
P_disease = 0.08  # Base rate (prior)
P_healthy = 1 - P_disease

# Test characteristics
sensitivity = 0.92  # P(positive | disease)
specificity = 0.90  # P(negative | healthy)
false_positive_rate = 1 - specificity  # P(positive | healthy)

# Calculate P(positive test)
P_positive = (sensitivity * P_disease + false_positive_rate * P_healthy)

# Bayes' theorem: P(disease | positive)
P_disease_given_positive = pf.bayes_theorem(sensitivity, P_disease, P_positive)

print("Disease Diagnosis Analysis")
print("="*70)
print(f"\n1. PRIOR INFORMATION:")
print(f"   Base rate (prevalence): {P_disease:.0%}")
print(f"   Test sensitivity: {sensitivity:.0%}")
print(f"   Test specificity: {specificity:.0%}")

print(f"\n2. CALCULATED PROBABILITIES:")
print(f"   P(positive test) = {P_positive:.4f}")
print(f"   P(disease | positive test) = {P_disease_given_positive:.4f} = {P_disease_given_positive:.1%}")

print(f"\n3. INTERPRETATION:")
print(f"   If a field tests positive, there's a {P_disease_given_positive:.0%} chance")
print(f"   it actually has the disease.")
print(f"\n   ⚠️  Despite 92% test accuracy, only {P_disease_given_positive:.0%} of positive")
print(f"   tests indicate true disease! Why? Low base rate (8%).")

In [ ]:
# Visualize with population of 1000 fields
n_fields = 1000
n_diseased = int(n_fields * P_disease)
n_healthy = n_fields - n_diseased

# Test results
true_positives = int(n_diseased * sensitivity)
false_negatives = n_diseased - true_positives
false_positives = int(n_healthy * false_positive_rate)
true_negatives = n_healthy - false_positives

# Create confusion matrix
confusion = pd.DataFrame([
    [true_positives, false_positives, true_positives + false_positives],
    [false_negatives, true_negatives, false_negatives + true_negatives],
    [n_diseased, n_healthy, n_fields]
], 
    index=['Positive Test', 'Negative Test', 'Total'],
    columns=['Disease', 'Healthy', 'Total']
)

print("\nConfusion Matrix (per 1000 fields):")
print(confusion)

# Visualize
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))

# Population breakdown
categories = ['True\nPositive', 'False\nPositive', 'False\nNegative', 'True\nNegative']
values = [true_positives, false_positives, false_negatives, true_negatives]
colors = ['darkgreen', 'orange', 'red', 'lightgreen']

bars = ax1.bar(categories, values, color=colors, edgecolor='black', linewidth=2)
ax1.set_ylabel('Number of Fields')
ax1.set_title(f'Test Results Breakdown (n={n_fields})')
for i, v in enumerate(values):
    ax1.text(i, v + 5, str(v), ha='center', fontweight='bold')

# Among positive tests
n_positive = true_positives + false_positives
labels = [f'True Positive\n{true_positives}', f'False Positive\n{false_positives}']
sizes = [true_positives, false_positives]
colors_pie = ['darkgreen', 'orange']
explode = (0.1, 0)

ax2.pie(sizes, labels=labels, colors=colors_pie, autopct='%1.1f%%',
       startangle=90, explode=explode, textprops={'fontsize': 11, 'fontweight': 'bold'})
ax2.set_title(f'Among {n_positive} Positive Tests\nP(Disease|Positive) = {P_disease_given_positive:.0%}')

plt.tight_layout()
plt.show()

print(f"\n💡 Key Insight: Of {n_positive} positive tests, only {true_positives} are true diseases!")

## Problem 2: Cost-Benefit Decision Analysis

In [ ]:
# Economic parameters
treatment_cost = 150  # $/acre
crop_loss = 800  # $/acre if diseased and not treated
acres_per_field = 50

print("Decision Analysis: To Treat or Not to Treat?")
print("="*70)

# Scenario 1: Field tests POSITIVE
print(f"\nSCENARIO 1: Field tests POSITIVE (P(disease) = {P_disease_given_positive:.1%})")
print("-"*70)

# Option A: Treat
cost_treat = treatment_cost * acres_per_field
print(f"\nOption A - TREAT:")
print(f"  Cost: ${cost_treat:,} (certain)")

# Option B: Don't treat
expected_loss_no_treat = P_disease_given_positive * crop_loss * acres_per_field
print(f"\nOption B - DON'T TREAT:")
print(f"  Expected loss: {P_disease_given_positive:.1%} × ${crop_loss} × {acres_per_field} acres")
print(f"                = ${expected_loss_no_treat:,.0f}")

savings_if_treat = expected_loss_no_treat - cost_treat
print(f"\nExpected savings from treatment: ${savings_if_treat:,.0f}")

if savings_if_treat > 0:
    print(f"✓ DECISION: TREAT (saves ${savings_if_treat:,.0f} on average)")
else:
    print(f"✗ DECISION: DON'T TREAT")

# Scenario 2: Field tests NEGATIVE
P_disease_given_negative = pf.bayes_theorem(
    1 - sensitivity,  # P(negative | disease)
    P_disease,
    1 - P_positive  # P(negative)
)

print(f"\n" + "="*70)
print(f"\nSCENARIO 2: Field tests NEGATIVE (P(disease) = {P_disease_given_negative:.2%})")
print("-"*70)

expected_loss_negative = P_disease_given_negative * crop_loss * acres_per_field
print(f"Expected loss if don't treat: ${expected_loss_negative:,.0f}")
print(f"Cost to treat: ${cost_treat:,}")

if expected_loss_negative > cost_treat:
    print(f"✓ DECISION: TREAT (despite negative test)")
else:
    print(f"✓ DECISION: DON'T TREAT (very low disease probability)")

## Problem 3: Multi-Symptom Diagnosis

Real diagnosis uses multiple symptoms, not just one test.

In [ ]:
# Multiple symptoms/indicators
symptoms = {
    'Leaf Yellowing': {'sensitivity': 0.75, 'specificity': 0.80},
    'Wilting': {'sensitivity': 0.82, 'specificity': 0.85},
    'Root Discoloration': {'sensitivity': 0.88, 'specificity': 0.92}
}

print("Multi-Symptom Diagnosis")
print("="*70)

# Scenario: All three symptoms present
print("\nScenario: Field shows ALL three symptoms")
print("-"*70)

# Start with prior
current_prob = P_disease
print(f"Prior probability: {current_prob:.2%}")

# Update with each symptom (assuming independence)
for symptom_name, char in symptoms.items():
    sens = char['sensitivity']
    spec = char['specificity']
    fpr = 1 - spec
    
    # P(symptom)
    p_symptom = sens * current_prob + fpr * (1 - current_prob)
    
    # Update using Bayes
    current_prob = (sens * current_prob) / p_symptom
    
    print(f"After observing {symptom_name}: {current_prob:.1%}")

print(f"\nFinal probability of disease: {current_prob:.1%}")
print(f"\n💡 Multiple symptoms dramatically increase diagnostic confidence!")
print(f"   From {P_disease:.0%} (base rate) to {current_prob:.0%} (after symptoms)")

# Decision with multiple symptoms
expected_loss_multi = current_prob * crop_loss * acres_per_field
print(f"\nExpected loss if no treatment: ${expected_loss_multi:,.0f}")
print(f"Treatment cost: ${cost_treat:,}")
print(f"\n✓ STRONG RECOMMENDATION: TREAT (very high disease probability)")

## Summary: Key Lessons

### 1. Base Rates Matter
- Low disease prevalence means most positive tests are false positives
- Always consider prior probability, not just test accuracy

### 2. Economic Decision Making
- Compare expected costs of treating vs. not treating
- Decision depends on:
  - Test results (posterior probability)
  - Treatment cost
  - Potential crop loss

### 3. Multiple Evidence
- Combining multiple symptoms/tests dramatically improves diagnosis
- Each piece of evidence updates our probability
- Bayesian updating is the mathematically correct way to combine information

### 4. Practical Applications
- Disease diagnosis and treatment decisions
- Quality control and testing
- Risk assessment
- Resource allocation

### Next Steps
Continue to: `weather_prediction.ipynb` for probability in forecasting!